In [66]:
import http.client
import json
import csv

conn = http.client.HTTPSConnection("api.themoviedb.org")
movie_id = 550
api_key = "c5b2d9b19dd2e779e735854fb4f2de79"
conn.request("GET", "/3/movie/{0}/credits?api_key={1}".format(movie_id, api_key))
r1 = conn.getresponse()

In [25]:
import http.client
import json
import csv


#############################################################################################################################
# cse6242 
# All instructions, code comments, etc. contained within this notebook are part of the assignment instructions.
# Portions of this file will auto-graded in Gradescope using different sets of parameters / data to ensure that values are not
# hard-coded.
#
# Instructions:  Implement all methods in this file that have a return
# value of 'NotImplemented'. See the documentation within each method for specific details, including
# the expected return value
#
# Helper Functions:
# You are permitted to write additional helper functions/methods or use additional instance variables within
# the `Graph` class or `TMDbAPIUtils` class so long as the originally included methods work as required.
#
# Use:
# The `Graph` class  is used to represent and store the data for the TMDb co-actor network graph.  This class must
# also provide some basic analytics, i.e., number of nodes, edges, and nodes with the highest degree.
#
# The `TMDbAPIUtils` class is used to retrieve Actor/Movie data using themoviedb.org API.  We have provided a few necessary methods
# to test your code w/ the API, e.g.: get_movie_cast(), get_movie_credits_for_person().  You may add additional
# methods and instance variables as desired (see Helper Functions).
#
# The data that you retrieve from the TMDb API is used to build your graph using the Graph class.  After you build your graph using the
# TMDb API data, use the Graph class write_edges_file & write_nodes_file methods to produce the separate nodes and edges
# .csv files for use with the Argo-Lite graph visualization tool.
#
# While building the co-actor graph, you will be required to write code to expand the graph by iterating
# through a portion of the graph nodes and finding similar artists using the TMDb API. We will not grade this code directly
# but will grade the resulting graph data in your Argo-Lite graph snapshot.
#
#############################################################################################################################


class Graph:

    # Do not modify
    def __init__(self, with_nodes_file=None, with_edges_file=None):
        """
        option 1:  init as an empty graph and add nodes
        option 2: init by specifying a path to nodes & edges files
        """
        self.nodes = []
        self.edges = []
        if with_nodes_file and with_edges_file:
            nodes_CSV = csv.reader(open(with_nodes_file))
            nodes_CSV = list(nodes_CSV)[1:]
            self.nodes = [(n[0], n[1]) for n in nodes_CSV]

            edges_CSV = csv.reader(open(with_edges_file))
            edges_CSV = list(edges_CSV)[1:]
            self.edges = [(e[0], e[1]) for e in edges_CSV]


    def add_node(self, id: str, name: str) -> None:
        """
        add a tuple (id, name) representing a node to self.nodes if it does not already exist
        The graph should not contain any duplicate nodes
        """
        if(id, name) not in self.nodes:
            self.nodes.append((id, name))
        #return NotImplemented


    def add_edge(self, source: str, target: str) -> None:
        """
        Add an edge between two nodes if it does not already exist.
        An edge is represented by a tuple containing two strings: e.g.: ('source', 'target').
        Where 'source' is the id of the source node and 'target' is the id of the target node
        e.g., for two nodes with ids 'a' and 'b' respectively, add the tuple ('a', 'b') to self.edges
        """
        edge = (source, target)
        edge_ud = (target, source) #undirected Edge
        if (edge not in self.edges) and (edge_ud not in self.edges) :
            self.edges.append(edge)
        
        # return NotImplemented


    def total_nodes(self) -> int:
        """
        Returns an integer value for the total number of nodes in the graph
        """
        return len(self.nodes)
        # return NotImplemented


    def total_edges(self) -> int:
        """
        Returns an integer value for the total number of edges in the graph
        """
        return len(self.edges)
        # return NotImplemented


    def max_degree_nodes(self) -> dict:
        """
        Return the node(s) with the highest degree
        Return multiple nodes in the event of a tie
        Format is a dict where the key is the node_id and the value is an integer for the node degree
        e.g. {'a': 8}
        or {'a': 22, 'b': 22}
        """
        for node in self.nodes:
            count=0
            for edge in self.edges:
                if edge[0]==node[0] or edge[1]==node[0]:
                    count+=1
            mydict[node[0]]=count
            maxdegree_node=max(mydict, key=mydict.get)
            maxdegree=mydict.get(maxdegree_node)
        for i in mydict:
            if mydict.get(i)==maxdegree:
                dict[i]=maxdegree
        return dict
        # return NotImplemented


    def print_nodes(self):
        """
        No further implementation required
        May be used for de-bugging if necessary
        """
        print(self.nodes)


    def print_edges(self):
        """
        No further implementation required
        May be used for de-bugging if necessary
        """
        print(self.edges)


    # Do not modify
    def write_edges_file(self, path="edges.csv")->None:
        """
        write all edges out as .csv
        :param path: string
        :return: None
        """
        edges_path = path
        edges_file = open(edges_path, 'w', encoding='utf-8')

        edges_file.write("source" + "," + "target" + "\n")

        for e in self.edges:
            edges_file.write(e[0] + "," + e[1] + "\n")

        edges_file.close()
        print("finished writing edges to csv")


    # Do not modify
    def write_nodes_file(self, path="nodes.csv")->None:
        """
        write all nodes out as .csv
        :param path: string
        :return: None
        """
        nodes_path = path
        nodes_file = open(nodes_path, 'w', encoding='utf-8')

        nodes_file.write("id,name" + "\n")
        for n in self.nodes:
            nodes_file.write(n[0] + "," + n[1] + "\n")
        nodes_file.close()
        print("finished writing nodes to csv")



class  TMDBAPIUtils:

    # Do not modify
    def __init__(self, api_key:str):
        self.api_key=api_key


    def get_movie_cast(self, movie_id:str, limit:int=None, exclude_ids:list=None) -> list:
        """
        Get the movie cast for a given movie id, with optional parameters to exclude an cast member
        from being returned and/or to limit the number of returned cast members
        documentation url: https://developers.themoviedb.org/3/movies/get-movie-credits

        :param integer movie_id: a movie_id
        :param integer limit: maximum number of returned cast members by their 'order' attribute
            e.g., limit=5 will attempt to return the 5 cast members having 'order' attribute values between 0-4
            If after excluding, there are fewer cast members than the specified limit, then return the remaining members
                (excluding the ones whose order values are outside the limit range). 
            If cast members with 'order' attribute in the specified limit range have been excluded, do not include more cast members to reach the limit.
            If after excluding, the limit is not specified, then return all remaining cast members."
            e.g., if limit=5 and the actor whose id corresponds to cast member with order=1 is to be excluded,
            return cast members with order values [0, 2, 3, 4], not [0, 2, 3, 4, 5]
        :param list exclude_ids: a list of ints containing ids (not cast_ids) of cast members  that should be excluded from the returned result
            e.g., if exclude_ids are [353, 455] then exclude these from any result.
        :rtype: list
            return a list of dicts, one dict per cast member with the following structure:
                [{'id': '97909' # the id of the cast member
                'character': 'John Doe' # the name of the character played
                'credit_id': '52fe4249c3a36847f8012927' # id of the credit, ...}, ... ]
                Note that this is an example of the structure of the list and some of the fields returned by the API.
                The result of the API call will include many more fields for each cast member.

        Important: the exclude_ids processing should occur prior to limiting output.
        """
        #global movie_Cast_list
        movie_Cast_list = []
        movie_Cast = []

        conn = http.client.HTTPSConnection("www.themoviedb.org")
        #conn.request("GET", "/3/movie/{0}/credits?api_key={1}".format(movie_id, self.api_key))
        conn.request("GET","https://api.themoviedb.org//3/movie/{0}/credits?api_key={1}".format(movie_id, self.api_key))
        response = conn.getresponse()
        #response = requests.get('https://api.themoviedb.org/3/movie/{0}/credits?api_key={1}'.format(movie_id, api_key))
        #print(response.read())
        #print(self.api_key)
        #print(movie_id)
        print(response.status, response.reason)
        movie_data_list = json.loads(response.read().decode())
        
        movie_Cast = movie_data_list['cast']
        for i in movie_Cast:
            #print(dictat)
            movie_Cast_shrinked = {key_i: i[key_i] for key_i in i.keys() & {'id','character','credit_id',"order"}}
            movie_Cast_list.append(movie_Cast_shrinked)
            #print(movie_Cast_list)

        if exclude_ids:
            #exlude ids first
            exclude_id = exclude_ids
            movie_Cast_list = [d for d in movie_Cast_list if d['id'] not in exclude_id ]
            
        if limit:
            movie_Cast_list = [d for d in movie_Cast_list if d["order"] < limit ]

        return movie_Cast_list
    def get_movie_credits_for_person(self, person_id:str, vote_avg_threshold:float=None)->list:
            
            
        """
        Using the TMDb API, get the movie credits for a person serving in a cast role
        documentation url: https://developers.themoviedb.org/3/people/get-person-movie-credits

        :param string person_id: the id of a person
        :param vote_avg_threshold: optional parameter to return the movie credit if it is >=
            the specified threshold.
            e.g., if the vote_avg_threshold is 5.0, then only return credits with a vote_avg >= 5.0
        :rtype: list
            return a list of dicts, one dict per movie credit with the following structure:
                [{'id': '97909' # the id of the movie credit
                'title': 'Long, Stock and Two Smoking Barrels' # the title (not original title) of the credit
                'vote_avg': 5.0 # the float value of the vote average value for the credit}, ... ]
        """
        global movie_Credit_list
        movie_Credit_list = []
        global movie_Credit
        movie_Credit = []
        
        print(self.api_key)
        print(person_id)

        conn = http.client.HTTPSConnection("www.themoviedb.org")
        conn.request("GET", "https://api.themoviedb.org/3/person/{0}/movie_credits?api_key={1}&language=en-US".format(person_id, self.api_key))
        #https://api.themoviedb.org/3/person/{0}/movie_credits?api_key={1}&language=en-US.format(person_id, self.api_key
        r2 = conn.getresponse()
        #print(r2.read())
        #print(r2.status, r2.reason)
        if r2.status == 200:
            movie_data1 = r2.read()
            movie_data1_list = json.loads(movie_data1.decode())
            movie_Credit = movie_data1_list['cast']
            #print(movie_Credit)
        for j in movie_Credit:
            #print(dictat)
            movie_Credit_shrinked = {key_i: j[key_i] for key_i in j.keys() & {'id','vote_average','title', 'order'}}
            movie_Credit_list.append(movie_Credit_shrinked)
            #print(movie_Cast_list)

        if vote_avg_threshold :
            abv_vote = [d for d in movie_Credit_list if d['vote_average'] >= vote_avg_threshold]
        else:
            abv_vote = movie_Credit_list


        return abv_vote

Actor_main_Graph = Graph()
api_key = "c5b2d9b19dd2e779e735854fb4f2de79"
tmdb_api_utils = TMDBAPIUtils(api_key)
#Res = tmdb_api_utils.get_movie_cast('24341', 3, [91533])
#print(Res)

Cast_details = tmdb_api_utils.get_movie_credits_for_person('2975',8.0)
print(Cast_details)
    

c5b2d9b19dd2e779e735854fb4f2de79
2975
[{'video': False, 'vote_average': 7.6, 'overview': 'Boyz n the Hood is the popular and successful film and social criticism from John Singleton about the conditions in South Central Los Angeles where teenagers are involved in gun fights and drug dealing on a daily basis.', 'release_date': '1991-07-12', 'title': 'Boyz n the Hood', 'adult': False, 'backdrop_path': '/4nBAP4CmC4012nJdxUc3pT4jp1.jpg', 'genre_ids': [18, 80], 'id': 650, 'original_language': 'en', 'original_title': 'Boyz n the Hood', 'poster_path': '/v4ox4aSCNT5vyLXl4Q71JiWwCXW.jpg', 'vote_count': 1273, 'popularity': 31.539, 'character': "Jason 'Furious' Styles", 'credit_id': '52fe4264c3a36847f801aeb5', 'order': 0}, {'adult': False, 'backdrop_path': '/Yotz5R36D7GbELyBQbM7Ot5Ay2.jpg', 'genre_ids': [80, 18], 'id': 22073, 'original_language': 'en', 'original_title': 'Hoodlum', 'overview': "In 1934, the second most lucrative business in New York City was running 'the numbers'. When Madam Queen

In [64]:
import http.client
import json
import csv


#############################################################################################################################
# cse6242 
# All instructions, code comments, etc. contained within this notebook are part of the assignment instructions.
# Portions of this file will auto-graded in Gradescope using different sets of parameters / data to ensure that values are not
# hard-coded.
#
# Instructions:  Implement all methods in this file that have a return
# value of 'NotImplemented'. See the documentation within each method for specific details, including
# the expected return value
#
# Helper Functions:
# You are permitted to write additional helper functions/methods or use additional instance variables within
# the `Graph` class or `TMDbAPIUtils` class so long as the originally included methods work as required.
#
# Use:
# The `Graph` class  is used to represent and store the data for the TMDb co-actor network graph.  This class must
# also provide some basic analytics, i.e., number of nodes, edges, and nodes with the highest degree.
#
# The `TMDbAPIUtils` class is used to retrieve Actor/Movie data using themoviedb.org API.  We have provided a few necessary methods
# to test your code w/ the API, e.g.: get_movie_cast(), get_movie_credits_for_person().  You may add additional
# methods and instance variables as desired (see Helper Functions).
#
# The data that you retrieve from the TMDb API is used to build your graph using the Graph class.  After you build your graph using the
# TMDb API data, use the Graph class write_edges_file & write_nodes_file methods to produce the separate nodes and edges
# .csv files for use with the Argo-Lite graph visualization tool.
#
# While building the co-actor graph, you will be required to write code to expand the graph by iterating
# through a portion of the graph nodes and finding similar artists using the TMDb API. We will not grade this code directly
# but will grade the resulting graph data in your Argo-Lite graph snapshot.
#
#############################################################################################################################


class Graph:

    # Do not modify
    def __init__(self, with_nodes_file=None, with_edges_file=None):
        """
        option 1:  init as an empty graph and add nodes
        option 2: init by specifying a path to nodes & edges files
        """
        self.nodes = []
        self.edges = []
        if with_nodes_file and with_edges_file:
            nodes_CSV = csv.reader(open(with_nodes_file))
            nodes_CSV = list(nodes_CSV)[1:]
            self.nodes = [(n[0], n[1]) for n in nodes_CSV]

            edges_CSV = csv.reader(open(with_edges_file))
            edges_CSV = list(edges_CSV)[1:]
            self.edges = [(e[0], e[1]) for e in edges_CSV]


    def add_node(self, id: str, name: str) -> None:
        """
        add a tuple (id, name) representing a node to self.nodes if it does not already exist
        The graph should not contain any duplicate nodes
        """
        if(id, name) not in self.nodes:
            self.nodes.append((id, name))
        #return NotImplemented


    def add_edge(self, source: str, target: str) -> None:
        """
        Add an edge between two nodes if it does not already exist.
        An edge is represented by a tuple containing two strings: e.g.: ('source', 'target').
        Where 'source' is the id of the source node and 'target' is the id of the target node
        e.g., for two nodes with ids 'a' and 'b' respectively, add the tuple ('a', 'b') to self.edges
        """
        edge = (source, target)
        edge_ud = (target, source) #undirected Edge
        if (edge not in self.edges) and (edge_ud not in self.edges) :
            self.edges.append(edge)
        
        # return NotImplemented


    def total_nodes(self) -> int:
        """
        Returns an integer value for the total number of nodes in the graph
        """
        return len(self.nodes)
        # return NotImplemented


    def total_edges(self) -> int:
        """
        Returns an integer value for the total number of edges in the graph
        """
        return len(self.edges)
        # return NotImplemented


    def max_degree_nodes(self) -> dict:
        """
        Return the node(s) with the highest degree
        Return multiple nodes in the event of a tie
        Format is a dict where the key is the node_id and the value is an integer for the node degree
        e.g. {'a': 8}
        or {'a': 22, 'b': 22}
        """
        for node in self.nodes:
            count=0
            for edge in self.edges:
                if edge[0]==node[0] or edge[1]==node[0]:
                    count+=1
            mydict[node[0]]=count
            maxdegree_node=max(mydict, key=mydict.get)
            maxdegree=mydict.get(maxdegree_node)
        for i in mydict:
            if mydict.get(i)==maxdegree:
                dict[i]=maxdegree
        return dict
        # return NotImplemented


    def print_nodes(self):
        """
        No further implementation required
        May be used for de-bugging if necessary
        """
        print(self.nodes)


    def print_edges(self):
        """
        No further implementation required
        May be used for de-bugging if necessary
        """
        print(self.edges)


    # Do not modify
    def write_edges_file(self, path="edges.csv")->None:
        """
        write all edges out as .csv
        :param path: string
        :return: None
        """
        edges_path = path
        edges_file = open(edges_path, 'w', encoding='utf-8')

        edges_file.write("source" + "," + "target" + "\n")

        for e in self.edges:
            edges_file.write(str(e[0]) + "," + str(e[1]) + "\n")

        edges_file.close()
        print("finished writing edges to csv")


    # Do not modify
    def write_nodes_file(self, path="nodes.csv")->None:
        """
        write all nodes out as .csv
        :param path: string
        :return: None
        """
        nodes_path = path
        nodes_file = open(nodes_path, 'w', encoding='utf-8')

        nodes_file.write("id,name" + "\n")
        for n in self.nodes:
            nodes_file.write(str(n[0]) + "," + str(n[1]) + "\n")
        nodes_file.close()
        print("finished writing nodes to csv")



class  TMDBAPIUtils:

    # Do not modify
    def __init__(self, api_key:str):
        self.api_key=api_key


    def get_movie_cast(self, movie_id:str, limit:int=None, exclude_ids:list=None) -> list:
        """
        Get the movie cast for a given movie id, with optional parameters to exclude an cast member
        from being returned and/or to limit the number of returned cast members
        documentation url: https://developers.themoviedb.org/3/movies/get-movie-credits

        :param integer movie_id: a movie_id
        :param integer limit: maximum number of returned cast members by their 'order' attribute
            e.g., limit=5 will attempt to return the 5 cast members having 'order' attribute values between 0-4
            If after excluding, there are fewer cast members than the specified limit, then return the remaining members
                (excluding the ones whose order values are outside the limit range). 
            If cast members with 'order' attribute in the specified limit range have been excluded, do not include more cast members to reach the limit.
            If after excluding, the limit is not specified, then return all remaining cast members."
            e.g., if limit=5 and the actor whose id corresponds to cast member with order=1 is to be excluded,
            return cast members with order values [0, 2, 3, 4], not [0, 2, 3, 4, 5]
        :param list exclude_ids: a list of ints containing ids (not cast_ids) of cast members  that should be excluded from the returned result
            e.g., if exclude_ids are [353, 455] then exclude these from any result.
        :rtype: list
            return a list of dicts, one dict per cast member with the following structure:
                [{'id': '97909' # the id of the cast member
                'character': 'John Doe' # the name of the character played
                'credit_id': '52fe4249c3a36847f8012927' # id of the credit, ...}, ... ]
                Note that this is an example of the structure of the list and some of the fields returned by the API.
                The result of the API call will include many more fields for each cast member.

        Important: the exclude_ids processing should occur prior to limiting output.
        """
        #global movie_Cast_list
        movie_Cast_list = []
        movie_Cast = []

        conn = http.client.HTTPSConnection("www.themoviedb.org")
        #conn.request("GET", "/3/movie/{0}/credits?api_key={1}".format(movie_id, self.api_key))
        conn.request("GET","https://api.themoviedb.org//3/movie/{0}/credits?api_key={1}".format(movie_id, self.api_key))
        response = conn.getresponse()
        #response = requests.get('https://api.themoviedb.org/3/movie/{0}/credits?api_key={1}'.format(movie_id, api_key))
        #print(response.read())
        #print(self.api_key)
        #print(movie_id)
        #print(response.status, response.reason)
        movie_data_list = json.loads(response.read().decode())
        
        movie_Cast = movie_data_list['cast']
        for i in movie_Cast:
            #print(dictat)
            movie_Cast_shrinked = {key_i: i[key_i] for key_i in i.keys() & {'id','character','credit_id',"order", "name"}}
            movie_Cast_list.append(movie_Cast_shrinked)
            #print(movie_Cast_list)

        if exclude_ids:
            #exlude ids first
            exclude_id = exclude_ids
            movie_Cast_list = [d for d in movie_Cast_list if d['id'] not in exclude_id ]
            
        if limit:
            movie_Cast_list = [d for d in movie_Cast_list if d["order"] < limit ]

        return movie_Cast_list
    def get_movie_credits_for_person(self, person_id:str, vote_avg_threshold:float=None)->list:
            
            
        """
        Using the TMDb API, get the movie credits for a person serving in a cast role
        documentation url: https://developers.themoviedb.org/3/people/get-person-movie-credits

        :param string person_id: the id of a person
        :param vote_avg_threshold: optional parameter to return the movie credit if it is >=
            the specified threshold.
            e.g., if the vote_avg_threshold is 5.0, then only return credits with a vote_avg >= 5.0
        :rtype: list
            return a list of dicts, one dict per movie credit with the following structure:
                [{'id': '97909' # the id of the movie credit
                'title': 'Long, Stock and Two Smoking Barrels' # the title (not original title) of the credit
                'vote_avg': 5.0 # the float value of the vote average value for the credit}, ... ]
        """
        #global movie_Credit_list
        movie_Credit_list = []
        #global movie_Credit
        movie_Credit = []
        
        #print(self.api_key)
        #print(person_id)

        conn = http.client.HTTPSConnection("www.themoviedb.org")
        conn.request("GET", "https://api.themoviedb.org/3/person/{0}/movie_credits?api_key={1}&language=en-US".format(person_id, self.api_key))
        #https://api.themoviedb.org/3/person/{0}/movie_credits?api_key={1}&language=en-US.format(person_id, self.api_key
        r2 = conn.getresponse()
        #print(r2.read())
        #print(r2.status, r2.reason)
        if r2.status == 200:
            movie_data1 = r2.read()
            movie_data1_list = json.loads(movie_data1.decode())
            movie_Credit = movie_data1_list['cast']
        for j in movie_Credit:
            #print(dictat)
            movie_Credit_shrinked = {key_i: j[key_i] for key_i in j.keys() & {'id','vote_average','title', 'order'}}
            movie_Credit_list.append(movie_Credit_shrinked)
            #print(movie_Cast_list)

        if vote_avg_threshold :
            abv_vote = [d for d in movie_Credit_list if d['vote_average'] >= vote_avg_threshold]
        else:
            abv_vote = movie_Credit_list


        return abv_vote
        
        #return NotImplemented


#############################################################################################################################
#
# BUILDING YOUR GRAPH
#
# Working with the API:  See use of http.request: https://docs.python.org/3/library/http.client.html#examples
#
# Using TMDb's API, build a co-actor network for the actor's/actress' highest rated movies
# In this graph, each node represents an actor
# An edge between any two nodes indicates that the two actors/actresses acted in a movie together
# i.e., they share a movie credit.!!!!!!!!!!!!!!!!!!
# e.g., An edge between Samuel L. Jackson and Robert Downey Jr. indicates that they have acted in one
# or more movies together.
#
# For this assignment, we are interested in a co-actor network of highly rated movies; specifically,
# we only want the top 3 co-actors in each movie credit of an actor having a vote average >= 8.0.
# Build your co-actor graph on the actor 'Laurence Fishburne' w/ person_id 2975.
#
# You will need to add extra functions or code to accomplish this.  We will not directly call or explicitly grade your
# algorithm. We will instead measure the correctness of your output by evaluating the data in your argo-lite graph
# snapshot.
#
# GRAPH SIZE
# With each iteration of your graph build, the number of nodes and edges grows approximately at an exponential rate.
# Our testing indicates growth approximately equal to e^2x.
# Since the TMDB API is a live database, the number of nodes / edges in the final graph will vary slightly depending on when
# you execute your graph building code. We take this into account by rebuilding the solution graph every few days and
# updating the auto-grader.  We establish a bound for lowest & highest encountered numbers of nodes and edges with a
# margin of +/- 100 for nodes and +/- 150 for edges.  e.g., The allowable range of nodes is set to:
#
# Min allowable nodes = min encountered nodes - 100
# Max allowable nodes = max allowable nodes + 100
#
# e.g., if the minimum encountered nodes = 507 and the max encountered nodes = 526, then the min/max range is 407-626
# The same method is used to calculate the edges with the exception of using the aforementioned edge margin.
# ----------------------------------------------------------------------------------------------------------------------
# BEGIN BUILD CO-ACTOR NETWORK
#
# INITIALIZE GRAPH
#   Initialize a Graph object with a single node representing Laurence Fishburne
#
# BEGIN BUILD BASE GRAPH:
#   Find all of Laurence Fishburne's movie credits that have a vote average >= 8.0
#   FOR each movie credit:
#   |   get the movie cast members having an 'order' value between 0-2 (these are the co-actors)
#   |
#   |   FOR each movie cast member:
#   |   |   using graph.add_node(), add the movie cast member as a node (keep track of all new nodes added to the graph)
#   |   |   using graph.add_edge(), add an edge between the Laurence Fishburne (actress) node
#   |   |   and each new node (co-actor/co-actress)
#   |   END FOR
#   END FOR
# END BUILD BASE GRAPH
#

#print(added_nodes)
# BEGIN LOOP - DO 2 TIMES:
#   IF first iteration of loop:
#   |   nodes = The nodes added in the BUILD BASE GRAPH (this excludes the original node of Laurence Fishburne!)
#   ELSE
#   |    nodes = The nodes added in the previous iteration:
#   ENDIF
#
#   FOR each node in nodes:
#   |  get the movie credits for the actor that have a vote average >= 8.0
#   |
#   |   FOR each movie credit:
#   |   |   try to get the 3 movie cast members having an 'order' value between 0-2
#   |   |
#   |   |   FOR each movie cast member:
#   |   |   |   IF the node doesn't already exist:
#   |   |   |   |    add the node to the graph (track all new nodes added to the graph)
#   |   |   |   ENDIF
#   |   |   |
#   |   |   |   IF the edge does not exist:
#   |   |   |   |   add an edge between the node (actor) and the new node (co-actor/co-actress)
#   |   |   |   ENDIF
#   |   |   END FOR
#   |   END FOR
#   END FOR
# END LOOP
#



# Your graph should not have any duplicate edges or nodes
# Write out your finished graph as a nodes file and an edges file using:
#   graph.write_edges_file()
#   graph.write_nodes_file()
#
# END BUILD CO-ACTOR NETWORK
# ----------------------------------------------------------------------------------------------------------------------

# Exception handling and best practices
# - You should use the param 'language=en-US' in all API calls to avoid encoding issues when writing data to file.
# - If the actor name has a comma char ',' it should be removed to prevent extra columns from being inserted into the .csv file
# - Some movie_credits may actually be collections and do not return cast data. Handle this situation by skipping these instances.
# - While The TMDb API does not have a rate-limiting scheme in place, consider that making hundreds / thousands of calls
#   can occasionally result in timeout errors. If you continue to experience 'ConnectionRefusedError : [Errno 61] Connection refused',
#   - wait a while and then try again.  It may be necessary to insert periodic sleeps when you are building your graph.


#for i in Cast_details:
#    movie_cast = tmdb_api_utils.get_movie_cast(i['id'],limit=3)
    
#movie_cast = tmdb_api_utils.get_movie_cast(i['id'],limit=3,exclude_ids=[2975])
#print(movie_cast)
#    for cast in movie_cast:
#        Actor_main_Graph.add_node(cast['id'],cast['name'])
#        Actor_main_Graph.add_edge(cast['id'],2975)
        
        
#count = 1

#exclude_list = [2975]


def return_name()->str:
    """
    Return a string containing your GT Username
    e.g., gburdell3
    Do not return your 9 digit GTId
    """
    
    return "zyang363"


def return_argo_lite_snapshot()->str:
    """
    Return the shared URL of your published graph in Argo-Lite
    """
    return NotImplemented



# You should modify __main__ as you see fit to build/test your graph using  the TMDBAPIUtils & Graph classes.
# Some boilerplate/sample code is provided for demonstration. We will not call __main__ during grading.

if __name__ == "__main__":

    My_Graph = Graph()
    My_Graph.add_node('2975','Laurence Fishburne')
    api_key = "c5b2d9b19dd2e779e735854fb4f2de79"
    tmdb_api_utils = TMDBAPIUtils(api_key)
    MVCredit = tmdb_api_utils.get_movie_credits_for_person('2975',8.0)
    exclude_list = []
    exclude_list.append("2975")
    added_nodes = []
    #print(exclude_list)
    for i in MVCredit:
        movie_cast = tmdb_api_utils.get_movie_cast(i['id'], limit=3,exclude_ids=exclude_list)
        #print(movie_cast)
        for cast in movie_cast:
            My_Graph.add_node(cast['id'], cast['name'])
            My_Graph.add_edge(cast['id'], "2975")
            exclude_list.append(cast['id'])
            added_nodes.append(cast['id'])
    
    for i in range(2):
    if i ==0:
        nodes = added_nodes
    else:
        nodes = new_added_nodes_list
    print(nodes)
    new_added_nodes_list = []
    for node in nodes: #go through each nodes
        #print(node)
        MVCredit_1 = tmdb_api_utils.get_movie_credits_for_person(str(node),8.0)
        exclude_list = []
        exclude_list.append(node)##### What if it link back to an existing node?
        for i in MVCredit_1:
            movie_cast_1 = tmdb_api_utils.get_movie_cast(i['id'], limit=3)
            for j in movie_cast_1: ##j is the 
                present_node = My_Graph.total_nodes()
                My_Graph.add_node(j['id'], j['name'])
                if My_Graph.total_nodes() > present_node: ##Check if the node has been added or not if it has, add to the list
                    new_added_nodes_list.append(j["id"])
                My_Graph.add_edge(j['id'], node)
    #print(new_added_nodes_list)

                    
                     # Node[0] is the source and j["id"] is the target for edges
    My_Graph.write_edges_file()
    My_Graph.write_nodes_file()
    #print(My_Graph.total_nodes())

    # If you have already built & written out your graph, you could read in your nodes & edges files
    # to perform testing on your graph.
    # graph = Graph(with_edges_file="edges.csv", with_nodes_file="nodes.csv")

[6384, 2975, 530, 1667883, 1232548, 1107983, 52057, 110380, 8349, 3084, 3087, 1776, 37872, 8328, 5141, 5139, 5140]
[18355, 64, 564, 584, 3219325, 3219326, 529, 532, 1954, 20959, 11521, 37625, 65731, 4937, 180659, 1280183, 181934, 2787630, 89289, 2102583, 73988, 2803017, 2954, 33663, 937663, 1931175, 1700943, 938972, 1236595, 1209864, 2685726, 2764421, 8654, 104053, 2535, 225620, 1038, 4173, 349, 4587, 9780, 19011, 146687, 1122373, 1122374, 1122375, 3203, 7862, 11367, 31028, 3266, 36059, 5606, 1179102, 21010, 101032, 928944, 2022250, 9979, 514, 6193, 1892, 7796, 851784, 125025, 122889, 2641, 72028, 4724, 66743, 1461, 2098136, 2098137, 9857, 5248, 1158, 1662371, 82702, 65605, 47775, 14386, 4483, 13570, 3417, 380, 106179, 10190, 5250, 12132, 2130, 3810, 72305, 44735, 3037, 4951, 8487, 8490, 6838, 1, 7879, 5064, 488, 2559324, 27888, 2303, 5293, 1032, 1150, 1243, 510, 55983, 7908, 134, 1006721, 3083483, 3, 14782, 190, 147, 2378014, 2046262, 1381997, 11784, 78021, 16847, 2714, 4955, 27995, 1

In [65]:
x = "shas,lo"

In [66]:
x.replace(",", "")

'shaslo'

In [67]:
import http.client
import json
import csv
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

#############################################################################################################################
# cse6242 
# All instructions, code comments, etc. contained within this notebook are part of the assignment instructions.
# Portions of this file will auto-graded in Gradescope using different sets of parameters / data to ensure that values are not
# hard-coded.
#
# Instructions:  Implement all methods in this file that have a return
# value of 'NotImplemented'. See the documentation within each method for specific details, including
# the expected return value
#
# Helper Functions:
# You are permitted to write additional helper functions/methods or use additional instance variables within
# the `Graph` class or `TMDbAPIUtils` class so long as the originally included methods work as required.
#
# Use:
# The `Graph` class  is used to represent and store the data for the TMDb co-actor network graph.  This class must
# also provide some basic analytics, i.e., number of nodes, edges, and nodes with the highest degree.
#
# The `TMDbAPIUtils` class is used to retrieve Actor/Movie data using themoviedb.org API.  We have provided a few necessary methods
# to test your code w/ the API, e.g.: get_movie_cast(), get_movie_credits_for_person().  You may add additional
# methods and instance variables as desired (see Helper Functions).
#
# The data that you retrieve from the TMDb API is used to build your graph using the Graph class.  After you build your graph using the
# TMDb API data, use the Graph class write_edges_file & write_nodes_file methods to produce the separate nodes and edges
# .csv files for use with the Argo-Lite graph visualization tool.
#
# While building the co-actor graph, you will be required to write code to expand the graph by iterating
# through a portion of the graph nodes and finding similar artists using the TMDb API. We will not grade this code directly
# but will grade the resulting graph data in your Argo-Lite graph snapshot.
#
#############################################################################################################################


class Graph:

    # Do not modify
    def __init__(self, with_nodes_file=None, with_edges_file=None):
        """
        option 1:  init as an empty graph and add nodes
        option 2: init by specifying a path to nodes & edges files
        """
        self.nodes = []
        self.edges = []
        if with_nodes_file and with_edges_file:
            nodes_CSV = csv.reader(open(with_nodes_file))
            nodes_CSV = list(nodes_CSV)[1:]
            self.nodes = [(n[0], n[1]) for n in nodes_CSV]

            edges_CSV = csv.reader(open(with_edges_file))
            edges_CSV = list(edges_CSV)[1:]
            self.edges = [(e[0], e[1]) for e in edges_CSV]


    def add_node(self, id: str, name: str) -> None:
        """
        add a tuple (id, name) representing a node to self.nodes if it does not already exist
        The graph should not contain any duplicate nodes
        """
        if "," in name:
            name = name.replace(",", " ")
        if(id, name) not in self.nodes:
            self.nodes.append((id, name))
        #return NotImplemented


    def add_edge(self, source: str, target: str) -> None:
        """
        Add an edge between two nodes if it does not already exist.
        An edge is represented by a tuple containing two strings: e.g.: ('source', 'target').
        Where 'source' is the id of the source node and 'target' is the id of the target node
        e.g., for two nodes with ids 'a' and 'b' respectively, add the tuple ('a', 'b') to self.edges
        """
        edge = (source, target)
        edge_ud = (target, source) #undirected Edge
        if (edge not in self.edges) and (edge_ud not in self.edges) and not(source == target):
            self.edges.append(edge)
        
        # return NotImplemented


    def total_nodes(self) -> int:
        """
        Returns an integer value for the total number of nodes in the graph
        """
        return len(self.nodes)
        # return NotImplemented


    def total_edges(self) -> int:
        """
        Returns an integer value for the total number of edges in the graph
        """
        return len(self.edges)
        # return NotImplemented


    def max_degree_nodes(self) -> dict:
        """
        Return the node(s) with the highest degree
        Return multiple nodes in the event of a tie
        Format is a dict where the key is the node_id and the value is an integer for the node degree
        e.g. {'a': 8}
        or {'a': 22, 'b': 22}
        """
        mydict = {}
        for node in self.nodes:
            count=0
            for edge in self.edges:
                if edge[0]==node[0] or edge[1]==node[0]:
                    count+=1
            mydict[node[0]]=count
            maxdegree_node=max(mydict, key=mydict.get)
            maxdegree=mydict.get(maxdegree_node)
        dict= {}
        for i in mydict:
            if mydict.get(i)==maxdegree:
                dict[i]=maxdegree
        return dict
        # return NotImplemented


    def print_nodes(self):
        """
        No further implementation required
        May be used for de-bugging if necessary
        """
        print(self.nodes)


    def print_edges(self):
        """
        No further implementation required
        May be used for de-bugging if necessary
        """
        print(self.edges)


    # Do not modify
    def write_edges_file(self, path="edges.csv")->None:
        """
        write all edges out as .csv
        :param path: string
        :return: None
        """
        edges_path = path
        edges_file = open(edges_path, 'w', encoding='utf-8')

        edges_file.write("source" + "," + "target" + "\n")

        for e in self.edges:
            edges_file.write(str(e[0]) + "," + str(e[1]) + "\n")

        edges_file.close()
        print("finished writing edges to csv")


    # Do not modify
    def write_nodes_file(self, path="nodes.csv")->None:
        """
        write all nodes out as .csv
        :param path: string
        :return: None
        """
        nodes_path = path
        nodes_file = open(nodes_path, 'w', encoding='utf-8')

        nodes_file.write("id,name" + "\n")
        for n in self.nodes:
            nodes_file.write(str(n[0]) + "," + str(n[1]) + "\n")
        nodes_file.close()
        print("finished writing nodes to csv")



class  TMDBAPIUtils:

    # Do not modify
    def __init__(self, api_key:str):
        self.api_key=api_key


    def get_movie_cast(self, movie_id:str, limit:int=None, exclude_ids:list=None) -> list:
        """
        Get the movie cast for a given movie id, with optional parameters to exclude an cast member
        from being returned and/or to limit the number of returned cast members
        documentation url: https://developers.themoviedb.org/3/movies/get-movie-credits

        :param integer movie_id: a movie_id
        :param integer limit: maximum number of returned cast members by their 'order' attribute
            e.g., limit=5 will attempt to return the 5 cast members having 'order' attribute values between 0-4
            If after excluding, there are fewer cast members than the specified limit, then return the remaining members
                (excluding the ones whose order values are outside the limit range). 
            If cast members with 'order' attribute in the specified limit range have been excluded, do not include more cast members to reach the limit.
            If after excluding, the limit is not specified, then return all remaining cast members."
            e.g., if limit=5 and the actor whose id corresponds to cast member with order=1 is to be excluded,
            return cast members with order values [0, 2, 3, 4], not [0, 2, 3, 4, 5]
        :param list exclude_ids: a list of ints containing ids (not cast_ids) of cast members  that should be excluded from the returned result
            e.g., if exclude_ids are [353, 455] then exclude these from any result.
        :rtype: list
            return a list of dicts, one dict per cast member with the following structure:
                [{'id': '97909' # the id of the cast member
                'character': 'John Doe' # the name of the character played
                'credit_id': '52fe4249c3a36847f8012927' # id of the credit, ...}, ... ]
                Note that this is an example of the structure of the list and some of the fields returned by the API.
                The result of the API call will include many more fields for each cast member.

        Important: the exclude_ids processing should occur prior to limiting output.
        """
        #global movie_Cast_list
        movie_Cast_list = []
        movie_Cast = []

        conn = http.client.HTTPSConnection("www.themoviedb.org")
        #conn.request("GET", "/3/movie/{0}/credits?api_key={1}".format(movie_id, self.api_key))
        conn.request("GET","https://api.themoviedb.org//3/movie/{0}/credits?api_key={1}".format(movie_id, self.api_key))
        response = conn.getresponse()
        #response = requests.get('https://api.themoviedb.org/3/movie/{0}/credits?api_key={1}'.format(movie_id, api_key))
        #print(response.read())
        #print(self.api_key)
        #print(movie_id)
        #print(response.status, response.reason)
        movie_data_list = json.loads(response.read().decode())
        
        movie_Cast = movie_data_list['cast']
        for i in movie_Cast:
            #print(dictat)
            movie_Cast_shrinked = {key_i: i[key_i] for key_i in i.keys() & {'id','character','credit_id',"order", "name"}}
            movie_Cast_list.append(movie_Cast_shrinked)
            #print(movie_Cast_list)

        if exclude_ids:
            #exlude ids first
            exclude_id = exclude_ids
            movie_Cast_list = [d for d in movie_Cast_list if d['id'] not in exclude_id ]
            
        if limit:
            movie_Cast_list = [d for d in movie_Cast_list if d["order"] < limit ]

        return movie_Cast_list
    def get_movie_credits_for_person(self, person_id:str, vote_avg_threshold:float=None)->list:
            
            
        """
        Using the TMDb API, get the movie credits for a person serving in a cast role
        documentation url: https://developers.themoviedb.org/3/people/get-person-movie-credits

        :param string person_id: the id of a person
        :param vote_avg_threshold: optional parameter to return the movie credit if it is >=
            the specified threshold.
            e.g., if the vote_avg_threshold is 5.0, then only return credits with a vote_avg >= 5.0
        :rtype: list
            return a list of dicts, one dict per movie credit with the following structure:
                [{'id': '97909' # the id of the movie credit
                'title': 'Long, Stock and Two Smoking Barrels' # the title (not original title) of the credit
                'vote_avg': 5.0 # the float value of the vote average value for the credit}, ... ]
        """
        
        movie_Credit_list = []
        movie_Credit = []
        
        #print(self.api_key)
        #print(person_id)

        conn = http.client.HTTPSConnection("www.themoviedb.org")
        conn.request("GET", "https://api.themoviedb.org/3/person/{0}/movie_credits?api_key={1}&language=en-US".format(person_id, self.api_key))
        #https://api.themoviedb.org/3/person/{0}/movie_credits?api_key={1}&language=en-US.format(person_id, self.api_key
        r2 = conn.getresponse()
        #print(r2.read())
        #print(r2.status, r2.reason)
        if r2.status == 200:
            movie_data1 = r2.read()
            movie_data1_list = json.loads(movie_data1.decode())
            movie_Credit = movie_data1_list['cast']
        for j in movie_Credit:
            #print(dictat)
            movie_Credit_shrinked = {key_i: j[key_i] for key_i in j.keys() & {'id','vote_average','title', 'order'}}
            movie_Credit_list.append(movie_Credit_shrinked)
            #print(movie_Cast_list)

        if vote_avg_threshold :
            abv_vote = [d for d in movie_Credit_list if d['vote_average'] >= vote_avg_threshold]
        else:
            abv_vote = movie_Credit_list


        return abv_vote
        
        #return NotImplemented


#############################################################################################################################
#
# BUILDING YOUR GRAPH
#
# Working with the API:  See use of http.request: https://docs.python.org/3/library/http.client.html#examples
#
# Using TMDb's API, build a co-actor network for the actor's/actress' highest rated movies
# In this graph, each node represents an actor
# An edge between any two nodes indicates that the two actors/actresses acted in a movie together
# i.e., they share a movie credit.!!!!!!!!!!!!!!!!!!
# e.g., An edge between Samuel L. Jackson and Robert Downey Jr. indicates that they have acted in one
# or more movies together.
#
# For this assignment, we are interested in a co-actor network of highly rated movies; specifically,
# we only want the top 3 co-actors in each movie credit of an actor having a vote average >= 8.0.
# Build your co-actor graph on the actor 'Laurence Fishburne' w/ person_id 2975.
#
# You will need to add extra functions or code to accomplish this.  We will not directly call or explicitly grade your
# algorithm. We will instead measure the correctness of your output by evaluating the data in your argo-lite graph
# snapshot.
#
# GRAPH SIZE
# With each iteration of your graph build, the number of nodes and edges grows approximately at an exponential rate.
# Our testing indicates growth approximately equal to e^2x.
# Since the TMDB API is a live database, the number of nodes / edges in the final graph will vary slightly depending on when
# you execute your graph building code. We take this into account by rebuilding the solution graph every few days and
# updating the auto-grader.  We establish a bound for lowest & highest encountered numbers of nodes and edges with a
# margin of +/- 100 for nodes and +/- 150 for edges.  e.g., The allowable range of nodes is set to:
#
# Min allowable nodes = min encountered nodes - 100
# Max allowable nodes = max allowable nodes + 100
#
# e.g., if the minimum encountered nodes = 507 and the max encountered nodes = 526, then the min/max range is 407-626
# The same method is used to calculate the edges with the exception of using the aforementioned edge margin.
# ----------------------------------------------------------------------------------------------------------------------
# BEGIN BUILD CO-ACTOR NETWORK
#
# INITIALIZE GRAPH
#   Initialize a Graph object with a single node representing Laurence Fishburne
#
# BEGIN BUILD BASE GRAPH:
#   Find all of Laurence Fishburne's movie credits that have a vote average >= 8.0
#   FOR each movie credit:
#   |   get the movie cast members having an 'order' value between 0-2 (these are the co-actors)
#   |
#   |   FOR each movie cast member:
#   |   |   using graph.add_node(), add the movie cast member as a node (keep track of all new nodes added to the graph)
#   |   |   using graph.add_edge(), add an edge between the Laurence Fishburne (actress) node
#   |   |   and each new node (co-actor/co-actress)
#   |   END FOR
#   END FOR
# END BUILD BASE GRAPH
#

#print(added_nodes)
# BEGIN LOOP - DO 2 TIMES:
#   IF first iteration of loop:
#   |   nodes = The nodes added in the BUILD BASE GRAPH (this excludes the original node of Laurence Fishburne!)
#   ELSE
#   |    nodes = The nodes added in the previous iteration:
#   ENDIF
#
#   FOR each node in nodes:
#   |  get the movie credits for the actor that have a vote average >= 8.0
#   |
#   |   FOR each movie credit:
#   |   |   try to get the 3 movie cast members having an 'order' value between 0-2
#   |   |
#   |   |   FOR each movie cast member:
#   |   |   |   IF the node doesn't already exist:
#   |   |   |   |    add the node to the graph (track all new nodes added to the graph)
#   |   |   |   ENDIF
#   |   |   |
#   |   |   |   IF the edge does not exist:
#   |   |   |   |   add an edge between the node (actor) and the new node (co-actor/co-actress)
#   |   |   |   ENDIF
#   |   |   END FOR
#   |   END FOR
#   END FOR
# END LOOP
#



# Your graph should not have any duplicate edges or nodes
# Write out your finished graph as a nodes file and an edges file using:
#   graph.write_edges_file()
#   graph.write_nodes_file()
#
# END BUILD CO-ACTOR NETWORK
# ----------------------------------------------------------------------------------------------------------------------

# Exception handling and best practices
# - You should use the param 'language=en-US' in all API calls to avoid encoding issues when writing data to file.
# - If the actor name has a comma char ',' it should be removed to prevent extra columns from being inserted into the .csv file
# - Some movie_credits may actually be collections and do not return cast data. Handle this situation by skipping these instances.
# - While The TMDb API does not have a rate-limiting scheme in place, consider that making hundreds / thousands of calls
#   can occasionally result in timeout errors. If you continue to experience 'ConnectionRefusedError : [Errno 61] Connection refused',
#   - wait a while and then try again.  It may be necessary to insert periodic sleeps when you are building your graph.


#for i in Cast_details:
#    movie_cast = tmdb_api_utils.get_movie_cast(i['id'],limit=3)
    
#movie_cast = tmdb_api_utils.get_movie_cast(i['id'],limit=3,exclude_ids=[2975])
#print(movie_cast)
#    for cast in movie_cast:
#        Actor_main_Graph.add_node(cast['id'],cast['name'])
#        Actor_main_Graph.add_edge(cast['id'],2975)
        
        
#count = 1

#exclude_list = [2975]


def return_name()->str:
    """
    Return a string containing your GT Username
    e.g., gburdell3
    Do not return your 9 digit GTId
    """
    
    return "zyang363"


def return_argo_lite_snapshot()->str:
    """
    Return the shared URL of your published graph in Argo-Lite
    """
    return NotImplemented



# You should modify __main__ as you see fit to build/test your graph using  the TMDBAPIUtils & Graph classes.
# Some boilerplate/sample code is provided for demonstration. We will not call __main__ during grading.

if __name__ == "__main__":

    My_Graph = Graph()
    My_Graph.add_node('2975','Laurence Fishburne')
    api_key = "c5b2d9b19dd2e779e735854fb4f2de79"
    tmdb_api_utils = TMDBAPIUtils(api_key)
    MVCredit = tmdb_api_utils.get_movie_credits_for_person('2975',8.0)
    exclude_list = []
    exclude_list.append("2975")
    added_nodes = []
    #print(exclude_list)
    for i in MVCredit:
        movie_cast = tmdb_api_utils.get_movie_cast(i['id'], limit=3,exclude_ids=exclude_list)
        #print(movie_cast)
        for cast in movie_cast:
            My_Graph.add_node(cast['id'], cast['name'])
            My_Graph.add_edge(cast['id'], "2975")
            exclude_list.append(cast['id'])
            added_nodes.append(cast['id'])
    
    for i in range(2):
        if i ==0:
            nodes = added_nodes
        else:
            nodes = new_added_nodes_list
        #print(nodes)
        new_added_nodes_list = []
        for node in nodes: #go through each nodes
            #print(node)
            MVCredit_1 = tmdb_api_utils.get_movie_credits_for_person(str(node),8.0)
            exclude_list = []
            exclude_list.append(node)##### What if it link back to an existing node?
            for i in MVCredit_1:
                movie_cast_1 = tmdb_api_utils.get_movie_cast(i['id'], limit=3)
                for j in movie_cast_1: ##j is the 
                    present_node = My_Graph.total_nodes()
                    My_Graph.add_node(j['id'], j['name'])
                    if My_Graph.total_nodes() > present_node: ##Check if the node has been added or not if it has, add to the list
                        new_added_nodes_list.append(j["id"])
                    My_Graph.add_edge(j['id'], node)
    #print(new_added_nodes_list)

                    
                     # Node[0] is the source and j["id"] is the target for edges
    My_Graph.write_edges_file()
    My_Graph.write_nodes_file()
    print(My_Graph.total_nodes())
    print(My_Graph.total_edges())
    My_Graph.print_nodes()

    # If you have already built & written out your graph, you could read in your nodes & edges files
    # to perform testing on your graph.
    # graph = Graph(with_edges_file="edges.csv", with_nodes_file="nodes.csv")

finished writing edges to csv
finished writing nodes to csv
1007
1620
[('2975', 'Laurence Fishburne'), (6384, 'Keanu Reeves'), (2975, 'Laurence Fishburne'), (530, 'Carrie-Anne Moss'), (1667883, 'Milo Gibson'), (1232548, 'Abigail Hawk'), (1107983, 'Martin Luther King'), (52057, 'Obba Babatundé'), (110380, 'Colin Powell'), (8349, 'Martin Sheen'), (3084, 'Marlon Brando'), (3087, 'Robert Duvall'), (1776, 'Francis Ford Coppola'), (37872, 'Eleanor Coppola'), (8328, 'John Milius'), (5141, 'Heather Langenkamp'), (5139, 'Robert Englund'), (5140, 'Wes Craven'), (18355, 'Alex Winter'), (64, 'Gary Oldman'), (564, 'Richard Linklater'), (584, 'Philip K. Dick'), (3219325, 'Luigi Marinus'), (3219326, 'Isaac McCarthy'), (529, 'Guy Pearce'), (532, 'Joe Pantoliano'), (1954, 'Billy Zane'), (20959, 'Louis Gossett Jr.'), (11521, 'Kristin Minter'), (37625, 'Andrew Garfield'), (65731, 'Sam Worthington'), (4937, 'Vince Vaughn'), (180659, 'Rosa Parks'), (1280183, 'Coretta Scott King'), (181934, 'Al Sharpton'), 

In [68]:
md = {"2":"20", "3":"30"}

In [70]:
maxdegree_node= max(md, key=md.get)

In [71]:
md.get(maxdegree_node)

'30'

In [87]:
len(md)

2

In [76]:
md[md.get()>2]

TypeError: get expected at least 1 arguments, got 0

In [83]:
newDict = dict()
# Iterate over all the items in dictionary and filter items which has even keys
for (key, value) in md.items():
   # Check if key is even then add pair to new dictionary
   if int(value) > 20:
       newDict[key] = value

In [84]:
newDict

{'3': '30'}

In [89]:
import http.client
import json
import csv
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

#############################################################################################################################
# cse6242 
# All instructions, code comments, etc. contained within this notebook are part of the assignment instructions.
# Portions of this file will auto-graded in Gradescope using different sets of parameters / data to ensure that values are not
# hard-coded.
#
# Instructions:  Implement all methods in this file that have a return
# value of 'NotImplemented'. See the documentation within each method for specific details, including
# the expected return value
#
# Helper Functions:
# You are permitted to write additional helper functions/methods or use additional instance variables within
# the `Graph` class or `TMDbAPIUtils` class so long as the originally included methods work as required.
#
# Use:
# The `Graph` class  is used to represent and store the data for the TMDb co-actor network graph.  This class must
# also provide some basic analytics, i.e., number of nodes, edges, and nodes with the highest degree.
#
# The `TMDbAPIUtils` class is used to retrieve Actor/Movie data using themoviedb.org API.  We have provided a few necessary methods
# to test your code w/ the API, e.g.: get_movie_cast(), get_movie_credits_for_person().  You may add additional
# methods and instance variables as desired (see Helper Functions).
#
# The data that you retrieve from the TMDb API is used to build your graph using the Graph class.  After you build your graph using the
# TMDb API data, use the Graph class write_edges_file & write_nodes_file methods to produce the separate nodes and edges
# .csv files for use with the Argo-Lite graph visualization tool.
#
# While building the co-actor graph, you will be required to write code to expand the graph by iterating
# through a portion of the graph nodes and finding similar artists using the TMDb API. We will not grade this code directly
# but will grade the resulting graph data in your Argo-Lite graph snapshot.
#
#############################################################################################################################


class Graph:

    # Do not modify
    def __init__(self, with_nodes_file=None, with_edges_file=None):
        """
        option 1:  init as an empty graph and add nodes
        option 2: init by specifying a path to nodes & edges files
        """
        self.nodes = []
        self.edges = []
        if with_nodes_file and with_edges_file:
            nodes_CSV = csv.reader(open(with_nodes_file))
            nodes_CSV = list(nodes_CSV)[1:]
            self.nodes = [(n[0], n[1]) for n in nodes_CSV]

            edges_CSV = csv.reader(open(with_edges_file))
            edges_CSV = list(edges_CSV)[1:]
            self.edges = [(e[0], e[1]) for e in edges_CSV]


    def add_node(self, id: str, name: str) -> None:
        """
        add a tuple (id, name) representing a node to self.nodes if it does not already exist
        The graph should not contain any duplicate nodes
        """
        if "," in name:
            name = name.replace(",", " ")
        if(id, name) not in self.nodes:
            self.nodes.append((id, name))
        #return NotImplemented


    def add_edge(self, source: str, target: str) -> None:
        """
        Add an edge between two nodes if it does not already exist.
        An edge is represented by a tuple containing two strings: e.g.: ('source', 'target').
        Where 'source' is the id of the source node and 'target' is the id of the target node
        e.g., for two nodes with ids 'a' and 'b' respectively, add the tuple ('a', 'b') to self.edges
        """
        edge = (source, target)
        edge_ud = (target, source) #undirected Edge
        if (edge not in self.edges) and (edge_ud not in self.edges) and not(source == target):
            self.edges.append(edge)
        
        # return NotImplemented


    def total_nodes(self) -> int:
        """
        Returns an integer value for the total number of nodes in the graph
        """
        return len(self.nodes)
        # return NotImplemented


    def total_edges(self) -> int:
        """
        Returns an integer value for the total number of edges in the graph
        """
        return len(self.edges)
        # return NotImplemented


    def max_degree_nodes(self) -> dict:
        """
        Return the node(s) with the highest degree
        Return multiple nodes in the event of a tie
        Format is a dict where the key is the node_id and the value is an integer for the node degree
        e.g. {'a': 8}
        or {'a': 22, 'b': 22}
        """
        mydict = {}
        for node in self.nodes:
            count=0
            for edge in self.edges:
                if edge[0]==node[0] or edge[1]==node[0]:
                    count+=1
            mydict[node[0]]=count
            maxdegree_node=max(mydict, key=mydict.get) ##Get max of the key (node)
            maxdegree=mydict.get(maxdegree_node) ## Get max of the edges value
        
        dict= {}
        for i in mydict:
            if mydict.get(i)==maxdegree:
                dict[i]=maxdegree
        return dict
        # return NotImplemented


    def print_nodes(self):
        """
        No further implementation required
        May be used for de-bugging if necessary
        """
        print(self.nodes)


    def print_edges(self):
        """
        No further implementation required
        May be used for de-bugging if necessary
        """
        print(self.edges)

    def count_nonleaf(self):
        mydict = {}
        for node in self.nodes:
            count=0
            for edge in self.edges:
                if edge[0]==node[0] or edge[1]==node[0]:
                    count+=1
            mydict[node[0]]=count
        newDict = dict()
        for (key, value) in mydict.items():
            if int(value) > 1:
                newDict[key] = value
        return len(newDict)
        

    # Do not modify
    def write_edges_file(self, path="edges.csv")->None:
        """
        write all edges out as .csv
        :param path: string
        :return: None
        """
        edges_path = path
        edges_file = open(edges_path, 'w', encoding='utf-8')

        edges_file.write("source" + "," + "target" + "\n")

        for e in self.edges:
            edges_file.write(str(e[0]) + "," + str(e[1]) + "\n")

        edges_file.close()
        print("finished writing edges to csv")


    # Do not modify
    def write_nodes_file(self, path="nodes.csv")->None:
        """
        write all nodes out as .csv
        :param path: string
        :return: None
        """
        nodes_path = path
        nodes_file = open(nodes_path, 'w', encoding='utf-8')

        nodes_file.write("id,name" + "\n")
        for n in self.nodes:
            nodes_file.write(str(n[0]) + "," + str(n[1]) + "\n")
        nodes_file.close()
        print("finished writing nodes to csv")



class  TMDBAPIUtils:

    # Do not modify
    def __init__(self, api_key:str):
        self.api_key=api_key


    def get_movie_cast(self, movie_id:str, limit:int=None, exclude_ids:list=None) -> list:
        """
        Get the movie cast for a given movie id, with optional parameters to exclude an cast member
        from being returned and/or to limit the number of returned cast members
        documentation url: https://developers.themoviedb.org/3/movies/get-movie-credits

        :param integer movie_id: a movie_id
        :param integer limit: maximum number of returned cast members by their 'order' attribute
            e.g., limit=5 will attempt to return the 5 cast members having 'order' attribute values between 0-4
            If after excluding, there are fewer cast members than the specified limit, then return the remaining members
                (excluding the ones whose order values are outside the limit range). 
            If cast members with 'order' attribute in the specified limit range have been excluded, do not include more cast members to reach the limit.
            If after excluding, the limit is not specified, then return all remaining cast members."
            e.g., if limit=5 and the actor whose id corresponds to cast member with order=1 is to be excluded,
            return cast members with order values [0, 2, 3, 4], not [0, 2, 3, 4, 5]
        :param list exclude_ids: a list of ints containing ids (not cast_ids) of cast members  that should be excluded from the returned result
            e.g., if exclude_ids are [353, 455] then exclude these from any result.
        :rtype: list
            return a list of dicts, one dict per cast member with the following structure:
                [{'id': '97909' # the id of the cast member
                'character': 'John Doe' # the name of the character played
                'credit_id': '52fe4249c3a36847f8012927' # id of the credit, ...}, ... ]
                Note that this is an example of the structure of the list and some of the fields returned by the API.
                The result of the API call will include many more fields for each cast member.

        Important: the exclude_ids processing should occur prior to limiting output.
        """
        #global movie_Cast_list
        movie_Cast_list = []
        movie_Cast = []

        conn = http.client.HTTPSConnection("www.themoviedb.org")
        #conn.request("GET", "/3/movie/{0}/credits?api_key={1}".format(movie_id, self.api_key))
        conn.request("GET","https://api.themoviedb.org//3/movie/{0}/credits?api_key={1}".format(movie_id, self.api_key))
        response = conn.getresponse()
        #response = requests.get('https://api.themoviedb.org/3/movie/{0}/credits?api_key={1}'.format(movie_id, api_key))
        #print(response.read())
        #print(self.api_key)
        #print(movie_id)
        #print(response.status, response.reason)
        movie_data_list = json.loads(response.read().decode())
        
        movie_Cast = movie_data_list['cast']
        for i in movie_Cast:
            #print(dictat)
            movie_Cast_shrinked = {key_i: i[key_i] for key_i in i.keys() & {'id','character','credit_id',"order", "name"}}
            movie_Cast_list.append(movie_Cast_shrinked)
            #print(movie_Cast_list)

        if exclude_ids:
            #exlude ids first
            exclude_id = exclude_ids
            movie_Cast_list = [d for d in movie_Cast_list if d['id'] not in exclude_id ]
            
        if limit:
            movie_Cast_list = [d for d in movie_Cast_list if d["order"] < limit ]

        return movie_Cast_list
    def get_movie_credits_for_person(self, person_id:str, vote_avg_threshold:float=None)->list:
            
            
        """
        Using the TMDb API, get the movie credits for a person serving in a cast role
        documentation url: https://developers.themoviedb.org/3/people/get-person-movie-credits

        :param string person_id: the id of a person
        :param vote_avg_threshold: optional parameter to return the movie credit if it is >=
            the specified threshold.
            e.g., if the vote_avg_threshold is 5.0, then only return credits with a vote_avg >= 5.0
        :rtype: list
            return a list of dicts, one dict per movie credit with the following structure:
                [{'id': '97909' # the id of the movie credit
                'title': 'Long, Stock and Two Smoking Barrels' # the title (not original title) of the credit
                'vote_avg': 5.0 # the float value of the vote average value for the credit}, ... ]
        """
        
        movie_Credit_list = []
        movie_Credit = []
        
        #print(self.api_key)
        #print(person_id)

        conn = http.client.HTTPSConnection("www.themoviedb.org")
        conn.request("GET", "https://api.themoviedb.org/3/person/{0}/movie_credits?api_key={1}&language=en-US".format(person_id, self.api_key))
        #https://api.themoviedb.org/3/person/{0}/movie_credits?api_key={1}&language=en-US.format(person_id, self.api_key
        r2 = conn.getresponse()
        #print(r2.read())
        #print(r2.status, r2.reason)
        if r2.status == 200:
            movie_data1 = r2.read()
            movie_data1_list = json.loads(movie_data1.decode())
            movie_Credit = movie_data1_list['cast']
        for j in movie_Credit:
            #print(dictat)
            movie_Credit_shrinked = {key_i: j[key_i] for key_i in j.keys() & {'id','vote_average','title', 'order'}}
            movie_Credit_list.append(movie_Credit_shrinked)
            #print(movie_Cast_list)

        if vote_avg_threshold :
            abv_vote = [d for d in movie_Credit_list if d['vote_average'] >= vote_avg_threshold]
        else:
            abv_vote = movie_Credit_list


        return abv_vote
        
        #return NotImplemented


#############################################################################################################################
#
# BUILDING YOUR GRAPH
#
# Working with the API:  See use of http.request: https://docs.python.org/3/library/http.client.html#examples
#
# Using TMDb's API, build a co-actor network for the actor's/actress' highest rated movies
# In this graph, each node represents an actor
# An edge between any two nodes indicates that the two actors/actresses acted in a movie together
# i.e., they share a movie credit.!!!!!!!!!!!!!!!!!!
# e.g., An edge between Samuel L. Jackson and Robert Downey Jr. indicates that they have acted in one
# or more movies together.
#
# For this assignment, we are interested in a co-actor network of highly rated movies; specifically,
# we only want the top 3 co-actors in each movie credit of an actor having a vote average >= 8.0.
# Build your co-actor graph on the actor 'Laurence Fishburne' w/ person_id 2975.
#
# You will need to add extra functions or code to accomplish this.  We will not directly call or explicitly grade your
# algorithm. We will instead measure the correctness of your output by evaluating the data in your argo-lite graph
# snapshot.
#
# GRAPH SIZE
# With each iteration of your graph build, the number of nodes and edges grows approximately at an exponential rate.
# Our testing indicates growth approximately equal to e^2x.
# Since the TMDB API is a live database, the number of nodes / edges in the final graph will vary slightly depending on when
# you execute your graph building code. We take this into account by rebuilding the solution graph every few days and
# updating the auto-grader.  We establish a bound for lowest & highest encountered numbers of nodes and edges with a
# margin of +/- 100 for nodes and +/- 150 for edges.  e.g., The allowable range of nodes is set to:
#
# Min allowable nodes = min encountered nodes - 100
# Max allowable nodes = max allowable nodes + 100
#
# e.g., if the minimum encountered nodes = 507 and the max encountered nodes = 526, then the min/max range is 407-626
# The same method is used to calculate the edges with the exception of using the aforementioned edge margin.
# ----------------------------------------------------------------------------------------------------------------------
# BEGIN BUILD CO-ACTOR NETWORK
#
# INITIALIZE GRAPH
#   Initialize a Graph object with a single node representing Laurence Fishburne
#
# BEGIN BUILD BASE GRAPH:
#   Find all of Laurence Fishburne's movie credits that have a vote average >= 8.0
#   FOR each movie credit:
#   |   get the movie cast members having an 'order' value between 0-2 (these are the co-actors)
#   |
#   |   FOR each movie cast member:
#   |   |   using graph.add_node(), add the movie cast member as a node (keep track of all new nodes added to the graph)
#   |   |   using graph.add_edge(), add an edge between the Laurence Fishburne (actress) node
#   |   |   and each new node (co-actor/co-actress)
#   |   END FOR
#   END FOR
# END BUILD BASE GRAPH
#

#print(added_nodes)
# BEGIN LOOP - DO 2 TIMES:
#   IF first iteration of loop:
#   |   nodes = The nodes added in the BUILD BASE GRAPH (this excludes the original node of Laurence Fishburne!)
#   ELSE
#   |    nodes = The nodes added in the previous iteration:
#   ENDIF
#
#   FOR each node in nodes:
#   |  get the movie credits for the actor that have a vote average >= 8.0
#   |
#   |   FOR each movie credit:
#   |   |   try to get the 3 movie cast members having an 'order' value between 0-2
#   |   |
#   |   |   FOR each movie cast member:
#   |   |   |   IF the node doesn't already exist:
#   |   |   |   |    add the node to the graph (track all new nodes added to the graph)
#   |   |   |   ENDIF
#   |   |   |
#   |   |   |   IF the edge does not exist:
#   |   |   |   |   add an edge between the node (actor) and the new node (co-actor/co-actress)
#   |   |   |   ENDIF
#   |   |   END FOR
#   |   END FOR
#   END FOR
# END LOOP
#



# Your graph should not have any duplicate edges or nodes
# Write out your finished graph as a nodes file and an edges file using:
#   graph.write_edges_file()
#   graph.write_nodes_file()
#
# END BUILD CO-ACTOR NETWORK
# ----------------------------------------------------------------------------------------------------------------------

# Exception handling and best practices
# - You should use the param 'language=en-US' in all API calls to avoid encoding issues when writing data to file.
# - If the actor name has a comma char ',' it should be removed to prevent extra columns from being inserted into the .csv file
# - Some movie_credits may actually be collections and do not return cast data. Handle this situation by skipping these instances.
# - While The TMDb API does not have a rate-limiting scheme in place, consider that making hundreds / thousands of calls
#   can occasionally result in timeout errors. If you continue to experience 'ConnectionRefusedError : [Errno 61] Connection refused',
#   - wait a while and then try again.  It may be necessary to insert periodic sleeps when you are building your graph.


#for i in Cast_details:
#    movie_cast = tmdb_api_utils.get_movie_cast(i['id'],limit=3)
    
#movie_cast = tmdb_api_utils.get_movie_cast(i['id'],limit=3,exclude_ids=[2975])
#print(movie_cast)
#    for cast in movie_cast:
#        Actor_main_Graph.add_node(cast['id'],cast['name'])
#        Actor_main_Graph.add_edge(cast['id'],2975)
        
        
#count = 1

#exclude_list = [2975]


def return_name()->str:
    """
    Return a string containing your GT Username
    e.g., gburdell3
    Do not return your 9 digit GTId
    """
    
    return "zyang363"


def return_argo_lite_snapshot()->str:
    """
    Return the shared URL of your published graph in Argo-Lite
    """
    URL1 = "https://poloclub.github.io/argo-graph-lite/#5df6b913-157c-4a9c-ae27-f1659b491543"
    return URL1



# You should modify __main__ as you see fit to build/test your graph using  the TMDBAPIUtils & Graph classes.
# Some boilerplate/sample code is provided for demonstration. We will not call __main__ during grading.

if __name__ == "__main__":

    My_Graph = Graph()
    My_Graph.add_node('2975','Laurence Fishburne')
    api_key = "c5b2d9b19dd2e779e735854fb4f2de79"
    tmdb_api_utils = TMDBAPIUtils(api_key)
    MVCredit = tmdb_api_utils.get_movie_credits_for_person('2975',8.0)
    exclude_list = []
    exclude_list.append("2975")
    added_nodes = []
    #print(exclude_list)
    for i in MVCredit:
        movie_cast = tmdb_api_utils.get_movie_cast(i['id'], limit=3,exclude_ids=exclude_list)
        #print(movie_cast)
        for cast in movie_cast:
            My_Graph.add_node(cast['id'], cast['name'])
            My_Graph.add_edge(cast['id'], "2975")
            exclude_list.append(cast['id'])
            added_nodes.append(cast['id'])
    
    for i in range(2):
        if i ==0:
            nodes = added_nodes
        else:
            nodes = new_added_nodes_list
        #print(nodes)
        new_added_nodes_list = []
        for node in nodes: #go through each nodes
            #print(node)
            MVCredit_1 = tmdb_api_utils.get_movie_credits_for_person(str(node),8.0)
            #exclude_list = []
            #exclude_list.append(node)##### What if it link back to an existing node?
            for i in MVCredit_1:
                movie_cast_1 = tmdb_api_utils.get_movie_cast(i['id'], limit=3)
                for j in movie_cast_1: ##j is the 
                    present_node = My_Graph.total_nodes()
                    My_Graph.add_node(j['id'], j['name'])
                    if My_Graph.total_nodes() != present_node: ##Check if the node has been added or not if it has, add to the list
                        new_added_nodes_list.append(j["id"])
                    My_Graph.add_edge(j['id'], node)
    #print(new_added_nodes_list)

                    
                     # Node[0] is the source and j["id"] is the target for edges
    My_Graph.write_edges_file()
    My_Graph.write_nodes_file()
    #print(My_Graph.total_nodes())
    #print(My_Graph.total_edges())
    #My_Graph.print_nodes()
    #print(My_Graph.count_nonleaf())

    # If you have already built & written out your graph, you could read in your nodes & edges files
    # to perform testing on your graph.
    # graph = Graph(with_edges_file="edges.csv", with_nodes_file="nodes.csv")

finished writing edges to csv
finished writing nodes to csv


In [90]:
res = 'sadsads!!!!nkjnsd'

's'

In [95]:
di = {}

In [99]:
di["0"] = 1

In [100]:
di["1"] = 2

In [101]:
di

{'0': 1, '1': 2}

In [104]:
r = di.get("0")
print(r)

1
